In [6]:
from tensorflow.python.keras.applications.vgg16 import VGG16
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D, Flatten
from tensorflow.python.keras.layers import Dense, Dropout, Input
from tensorflow.python.keras.models import Model

# 사전 학습된 모델 불러오기
# input_tensor = Input(shape=(150,150,3))
model = VGG16(weights='imagenet', include_top=True) #, input_tensor = input_tensor)

In [7]:
# 모델 Layer 데이터화
layer_dict = dict([(layer.name, layer) for layer in model.layers])

# Layer 추가
x = layer_dict['block5_pool'].output
# Cov2D Layer +
x = Conv2D(filters = 64, kernel_size=(3, 3), activation='relu')(x)
# MaxPooling2D Layer +
x = MaxPooling2D(pool_size=(2, 2))(x)
# Flatten Layer +
x = Flatten()(x)
# FC Layer +
x = Dense(2048, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(4, activation='softmax')(x)

# new model 정의
new_model = Model(inputs = model.input, outputs = x)

In [8]:
# CNN Pre-trained 가중치를 그대로 사용할때
for layer in new_model.layers[:19] :
    layer.trainable = False

new_model.summary()

# 컴파일 옵션
new_model.compile(loss='categorical_crossentropy',
                     optimizer='adam',
                     metrics=['accuracy'])

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [9]:
# 테스트 데이터 (고양이/강아지)
train_dir = r'C:\Users\jam2208\Documents\007_Hackerthon\dataset\status\train'
test_dir = r'C:\Users\jam2208\Documents\007_Hackerthon\dataset\status\test'

# 폴더에 따라 자동 분류
train_image_generator = ImageDataGenerator(rescale=1./255)
test_image_generator = ImageDataGenerator(rescale=1./255)

# 데이터 구조 생성
train_data_gen = train_image_generator.flow_from_directory(batch_size=10,
                                                           directory=train_dir,
                                                           shuffle=True,
                                                           target_size=(224, 224),
                                                           class_mode='categorical')

test_data_gen = test_image_generator.flow_from_directory(batch_size=10,
                                                         directory=test_dir,
                                                         target_size=(224, 224),
                                                         class_mode='categorical')

Found 800 images belonging to 4 classes.
Found 200 images belonging to 4 classes.


In [10]:
# 모델 학습
history = new_model.fit(train_data_gen, epochs=60,
                        validation_data=test_data_gen)

new_model.save("newVGG16_epochs80.h5")                        
                        
# 최종 결과 리포트
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(acc))

from matplotlib import pyplot as plt

plt.plot(epochs, acc, 'r', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='testing acc')
plt.title('Training and testing accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='testing loss')
plt.title('Training and testing loss')
plt.legend()

plt.show()

# 저장 모델 불러오기
# from keras.models import load_model

# new_model = load_model("newVGG16.h5")

Epoch 1/80
80/80 [==============================] - 112s 1s/step - loss: 1.2307 - accuracy: 0.4250 - val_loss: 0.7855 - val_accuracy: 0.6500
Epoch 2/80
80/80 [==============================] - 112s 1s/step - loss: 0.6408 - accuracy: 0.7337 - val_loss: 0.5617 - val_accuracy: 0.7450
Epoch 3/80
80/80 [==============================] - 107s 1s/step - loss: 0.4914 - accuracy: 0.7812 - val_loss: 0.5119 - val_accuracy: 0.7850
Epoch 4/80
80/80 [==============================] - 108s 1s/step - loss: 0.3808 - accuracy: 0.8487 - val_loss: 1.4041 - val_accuracy: 0.6500
Epoch 5/80
80/80 [==============================] - 110s 1s/step - loss: 0.4616 - accuracy: 0.8225 - val_loss: 0.5624 - val_accuracy: 0.8150
Epoch 6/80
80/80 [==============================] - 112s 1s/step - loss: 0.2572 - accuracy: 0.8900 - val_loss: 0.5214 - val_accuracy: 0.8300
Epoch 7/80
80/80 [==============================] - 111s 1s/step - loss: 0.2104 - accuracy: 0.9175 - val_loss: 0.5753 - val_accuracy: 0.8400
Epoch 8/80
80

KeyboardInterrupt: 